In [1]:
from qiskit import *
import random
import numpy as np

We'll use a two qubit circuit initialized wih a random state.

In [2]:
qc = QuantumCircuit(2)

ket = [random.random() for _ in range(4)]
N = np.sqrt(sum([amp**2 for amp in ket]))
ket = [amp/N for amp in ket]

qc.initialize(ket,qc.qregs[0])

Here's the state.

In [3]:
ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

(0.5688564481725127+0j)
(0.4348629989000199+0j)
(0.5902162467328289+0j)
(0.37274830067063636+0j)


Now let's apply a swap gate made out of `cx` gates. The effect should be to swap the middle amplitides.

In [4]:
qc.cx(0,1)
qc.cx(1,0)
qc.cx(0,1)

ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

(0.5688564481725127+0j)
(0.5902162467328289+0j)
(0.4348629989000199+0j)
(0.37274830067063636+0j)


The effect is exactly what we wanted. Now let's swap back in the same way. However, instead of `cx` for the middle gate, we'll use `crx`: the controlled `x` rotation.

In [5]:
qc.cx(0,1)
qc.crx(1,0)
qc.cx(0,1)

AttributeError: 'QuantumCircuit' object has no attribute 'crx'

**Looks like there is no `crx`. Which is strange, `cry` and `crz` exist!**

Here they are, as an example

In [6]:
qc.crz(np.pi,0,1)
qc.cry(np.pi,0,1)

AquaError: 'A qubit is expected for the control.'

These don't work, but not because `crz` and `cry` don't exist. **It is because these gates do not accept the use of integers to refer to qubits. Instead, we need to refer to them using the `QuantumRegister` object.**

In [7]:
qr = qc.qregs[0]

qc.crz(np.pi,qr[0],qr[1])
qc.cry(np.pi,qr[0],qr[1])

Now let's take a look at what the state is now.

In [8]:
ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

(0.5688564481725126+0j)
(0.5902162467328288+0j)
(0.4348629989000199+0j)
(-0.37274830067063647+0j)


Though `crx` doesn't exist, the multiply-controlled `mcrx` does.

We'll try again to make the swap using this. The angle $\pi$ is used, so that the effect should be the same as a `cx`.

In [9]:
qr = qc.qregs[0]

qc.cx(0,1)
qc.mcrx(np.pi,[qr[1]],qr[0])
qc.cx(0,1)

ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

(0.5688564481725127+0j)
-0.43486299890001984j
-0.5902162467328289j
(-0.3727483006706364+0j)


**This does have the swap effect, but it also introduces an unwanted phase on the middle two amplitudes.**

Now we'll apply this gate to each basis state to see the effect.

In [10]:
qc = QuantumCircuit(2)
ket = [1,0,0,0]
qc.initialize(ket,qc.qregs[0])

qc.mcrx(np.pi,[qr[0]],qr[1])

ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

(1+0j)
0j
0j
0j


In [11]:
qc = QuantumCircuit(2)
ket = [0,1,0,0]
qc.initialize(ket,qc.qregs[0])

qc.mcrx(np.pi,[qr[0]],qr[1])

ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

0j
0j
0j
-1j


In [12]:
qc = QuantumCircuit(2)
ket = [0,0,1,0]
qc.initialize(ket,qc.qregs[0])

qc.mcrx(np.pi,[qr[0]],qr[1])

ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

0j
0j
(1+0j)
0j


In [13]:
qc = QuantumCircuit(2)
ket = [0,0,0,1]
qc.initialize(ket,qc.qregs[0])

qc.mcrx(np.pi,[qr[0]],qr[1])

ket = execute(qc,Aer.get_backend('statevector_simulator')).result().get_statevector()
for amp in ket:
    print(amp)

0j
-1j
0j
0j


**It does the flips that it should, but with an additional phase of $-1$. This doesn't look right.**